In [22]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
plt.style.use('fivethirtyeight')

In [27]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import optuna
from datetime import datetime
import random
import itertools
import statsmodels.api as sm
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error,mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
from IPython.display import HTML
"----------------------------------------------------------------------лишнее удалить"

'----------------------------------------------------------------------лишнее удалить'

In [23]:
sber = pd.read_csv('Сбербанк_1day.txt', delimiter = ';')
sber = sber[['<DATE>', '<CLOSE>', '<VOLUME>','<OPEN>','<AMOUNT>']]
sber = sber.rename(columns={'<DATE>': 'Date', '<CLOSE>': 'Close_s', '<VOLUME>': 'Vol_s', '<OPEN>': 'Open_s', '<AMOUNT>': 'Amount_s'})

In [24]:
sber = pd.read_csv('Сбербанк_1day_05052022_03042024.txt', delimiter = ';')
sber = sber[['<DATE>', '<CLOSE>', '<VOLUME>','<OPEN>','<AMOUNT>']]
sber = sber.rename(columns={'<DATE>': 'Date', '<CLOSE>': 'Close_s', '<VOLUME>': 'Vol_s', '<OPEN>': 'Open_s', '<AMOUNT>': 'Amount_s'})

yandex = pd.read_csv('Yandex_clA_1day_05052022_03042024 (1).txt', delimiter = ';')
yandex = yandex[['<DATE>', '<CLOSE>', '<VOLUME>','<OPEN>','<AMOUNT>']]
yandex = yandex.rename(columns={'<DATE>': 'Date', '<CLOSE>': 'Close_y', '<VOLUME>': 'Vol_y', '<OPEN>': 'Open_y', '<AMOUNT>': 'Amount_y'})

cb = pd.read_excel('Инфляция и ключевая ставка Банка России_F17_09_2013_T02_04_2024.xlsx')
cb = cb[['Дата', 'Ключевая ставка, % годовых', 'Инфляция, % г/г']]
cb = cb.rename(columns={'Дата': 'Date', 'Ключевая ставка, % годовых': 'Ключевая ставка', 'Инфляция, % г/г': 'Инфляция'})

eur = pd.read_csv('eur.csv')
usd = pd.read_csv('usd.csv')
gbp = pd.read_csv('gbp.csv')

eur = eur.groupby('Date').agg({'Price':'mean'}).reset_index()
eur = eur.rename(columns={'Price': 'Price_eur'})
usd = usd.groupby('Date').agg({'Price':'mean'}).reset_index()
usd = usd.rename(columns={'Price': 'Price_usd'})
gbp = gbp.groupby('Date').agg({'Price':'mean'}).reset_index()
gbp = gbp.rename(columns={'Price': 'Price_gbp'})

In [25]:
"--------------------------------------деление на тест и не тест выборку, доп фичи"

'--------------------------------------деление на тест и не тест выборку, доп фичи'

In [26]:
def rangeDate(start_date, end_date):
    res = pd.date_range(
        min(start_date, end_date),
        max(start_date, end_date)
    ).strftime('%Y-%m-%d').tolist()
    return res

fstart = datetime.datetime.strptime('2022-05-05', '%Y-%m-%d')
fend = datetime.datetime.strptime('2024-03-29', '%Y-%m-%d')
start = datetime.datetime.combine(fstart, datetime.time(0,0))
end = datetime.datetime.combine(fend, datetime.time(23, 59))

df = pd.DataFrame()
df['Date'] = rangeDate(start, end)

df = df.merge(sber, on = "Date", how = 'right')
df = df.merge(usd, on="Date", how = 'right')
df = df.merge(yandex, on="Date", how = 'right')

df = df.bfill()
df.set_index('Date', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 2022-05-05 to 2024-04-03
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Close_s    699 non-null    float64
 1   Vol_s      699 non-null    float64
 2   Open_s     699 non-null    float64
 3   Amount_s   699 non-null    float64
 4   Price_usd  699 non-null    float64
 5   Close_y    700 non-null    float64
 6   Vol_y      700 non-null    float64
 7   Open_y     700 non-null    float64
 8   Amount_y   700 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 54.7+ KB


In [36]:
'''plt.figure(figsize=(15,8))
plt.plot(df['Open_s'])
plt.title('График временного ряда')
plt.xlabel('Дата')
plt.ylabel('pricee')
plt.show()'''

"plt.figure(figsize=(15,8))\nplt.plot(df['Open_s'])\nplt.title('График временного ряда')\nplt.xlabel('Дата')\nplt.ylabel('pricee')\nplt.show()"

In [8]:
'''def search_optimal_sarima(time_series, seasonal_cycle):
    order_vals = diff_vals = ma_vals = range(0, 2)
    pdq_combinations = list(itertools.product(order_vals, diff_vals, ma_vals))
    seasonal_combinations = [(combo[0], combo[1], combo[2], seasonal_cycle) for combo in pdq_combinations]
       
    smallest_aic = float("inf")
    optimal_order_param = optimal_seasonal_param = None

    for order_param in pdq_combinations:
        for seasonal_param in seasonal_combinations:
            try:
                sarima_model = sm.tsa.statespace.SARIMAX(time_series,
                                                         order=order_param,
                                                         seasonal_order=seasonal_param,
                                                         enforce_stationarity=False,
                                                         enforce_invertibility=False)

                model_results = sarima_model.fit()
                if model_results.aic < smallest_aic:
                    smallest_aic = model_results.aic
                    optimal_order_param = order_param
                    optimal_seasonal_param = seasonal_param
            except:
                continue

    print('ARIMA{}x{} - AIC:{}'.format(optimal_order_param, optimal_seasonal_param, smallest_aic))

seasonal_cycle_length = 7
search_optimal_sarima(df['Open_s'], seasonal_cycle_length)'''

ARIMA(0, 1, 1)x(0, 1, 1, 7) - AIC:3364.2985751608953


In [29]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(df['Open_s'], order=(0, 1, 1), seasonal_order=(0, 1, 1, 7))

In [30]:
results = model.fit()
#print(results.summary())
'''results.plot_diagnostics(figsize=(12, 8))
plt.show()'''

In [32]:
st_pred = results.get_prediction(start=pd.to_datetime('2023-02-01'), dynamic=False) 
forecast_values = st_pred.predicted_mean
actual_values = df['2023-02-01':]['Close_s']

print(results.get_prediction(start=pd.to_datetime('2024-10-24'), end=pd.to_datetime('2024-10-24') ).predicted_mean)

2024-10-24    351.576725
Freq: D, dtype: float64


In [33]:
def pred(period_start, period_end):   #'2023-02-01'
    st_pred = results.get_prediction(start=pd.to_datetime(period_start), end=pd.to_datetime(period_end) , dynamic=False) 
    forecast_values = st_pred.predicted_mean
    actual_values = df['2023-02-01':]['Close_s']
    return st_pred.predicted_mean